El siguiente bloque descarga todos los archivos necesarios para la correcta ejecución del cuaderno.

In [1]:
%%capture
!mkdir v002
!mkdir images
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Agent.py' -O v002/Agent.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment.py' -O v002/Enviroment.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/__init__.py' -O v002/__init__.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment_with_agents.py' -O v002/Enviroment_with_agents.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/InOut_Simple_Laberinth.py' -O v002/InOut_Simple_Laberinth.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelNoTomato.bmp' -O images/PixelNoTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelTomato.bmp' -O images/PixelTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/face1_borders.bmp' -O images/face1_borders.bmp
!wget 'https://raw.githubusercontent.com/juanvs00/AgentSurvival/main/rules_to_clips_v3.py' -O rules_to_clips_v3.py

from v002 import *
from rules_to_clips_v3 import *

!pip install clipspy

En el siguiente bloque se deben escribir las reglas CLIPS y el nombre del agente.
El puzzle a resolver será que un agente busque una salida en un escenario sin muros.

In [2]:
# TODO: Aquí debes escribir las reglas CLIPS
rules = []
rule1 = """
    (defrule giraDcha
    (not (walls $? right $?))
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    (not (caution_loop))
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_right()" "self.move_forward()"))
    )"""
rules.append(rule1)
rule2 = """
    (defrule haciaDelante
    (walls $? right $?)
    (not (walls $? front $?))
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    (not (caution_loop))
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )"""
rules.append(rule2)
rule3 = """
    (defrule giraIzqda
    (walls $? right $?)
    (walls $? front $?)
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    (not (caution_loop))
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_left()"))
    )"""
rules.append(rule3)
rule4 = """
  (defrule salir
  (available_object exit ?x)
  ?h<-(pending_function_calls $?calls)
  =>
  (retract ?h)
  (assert (pending_function_calls ?x))
  )"""
rules.append(rule4)
rule5 = """
  (defrule bucle
  ?l<-(caution_loop)
  ?h<-(pending_function_calls $?calls)
  =>
  (retract ?l)
  (retract ?h)
  (assert (pending_function_calls $?calls "self.turn_left()" "self.move_forward()"))
  )"""
rules.append(rule5)

# TODO: Aquí debes escribir el nombre de tu agente
agent_name = "YO!"

En el siguiente bloque se inicializan los tipos de agentes que vayas a utilizar.

In [3]:
runner = Agent_CLIPS(rules)

El siguiente bloque es la función move del agente, no modificar.

In [4]:
def move(self, available_objects=None):

  # Actualizar el entorno CLIPS y obtener las interacciones del agente
  walls, objects, pos, ori = runner.agent_context(self)

  # Obtener las funciones y los objetos gracias al algoritmo
  functions, available_objects = algorithm(objects, runner.env, available_objects)

  # Ejecutar las funciones en pending_function_calls
  for j in functions:
      j = j.replace('"','')
      refresh_position(runner.env,j)
      eval(j)

El siguiente bloque es la ejecución del puzzle.
Puedes modificar el primer parámetro de la función InOut_Simple_Laberinth, que corresponde con el tamaño del escenario.

In [ ]:
reset_position(runner.env, runner.memory)

lb1 = No_Walls_Laberinth(7, plot_run='every epoch', move_protection=False)
lb1.create_agent(agent_name, move)
lb1.run()

El siguiente bloque es la herramienta para comprobar cuántas veces es tu agente capaz de escapar del laberinto.
Puedes modificar la variable num_runs si lo deseas.

In [ ]:
from tqdm import tqdm
num_runs = 200
num_success =  0

for _ in tqdm(range(num_runs)):
  lb1 = No_Walls_Laberinth(7, plot_run='never', move_protection=False)
  lb1.create_agent(agent_name, move)
  winner = lb1.run()

  if winner is not None:
    num_success += 1

print("\n\nEncontraste la salida en", num_success, "ocasiones de", num_runs)